In [1]:
import shutil
import os
import re
import cv2
# will use them for creating custom directory iterator
import numpy as np
from six.moves import range
import json

from keras.models import Model
from keras.layers import Dense
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.preprocessing.image import DirectoryIterator, ImageDataGenerator
from keras.callbacks import CSVLogger, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TensorBoard
from keras import backend as K

from keras.applications.resnet50 import preprocess_input

Using TensorFlow backend.


In [3]:
# regular expression for splitting by whitespace
splitter = re.compile("\s+")
base_path = '/home/ec2-user/GitHub/deepfashion_keras/data/img'
create_dict_bboxes_path = '/home/ec2-user/GitHub/deepfashion_keras/data/'

In [6]:
def process_folders():
    # Read the relevant annotation file and preprocess it
    # Assumed that the annotation files are under '<project folder>/data/anno' path
    with open('/home/ec2-user/data/deepfashion/eval/list_eval_partition.txt', 'r') as eval_partition_file:
        list_eval_partition = [line.rstrip('\n') for line in eval_partition_file][2:]
        list_eval_partition = [splitter.split(line) for line in list_eval_partition]
        list_all = [(v[0][4:], v[0].split('/')[1].split('_')[-1], v[1]) for v in list_eval_partition]

    # Put each image into the relevant folder in train/test/validation folder
    for element in list_all:
        if not os.path.exists(os.path.join(base_path, element[2])):
            os.mkdir(os.path.join(base_path, element[2]))
        if not os.path.exists(os.path.join(os.path.join(base_path, element[2]), element[1])):
            os.mkdir(os.path.join(os.path.join(base_path, element[2]), element[1]))
        if not os.path.exists(os.path.join(os.path.join(os.path.join(os.path.join(base_path, element[2]), element[1])),
                              element[0].split('/')[0])):
            os.mkdir(os.path.join(os.path.join(os.path.join(os.path.join(base_path, element[2]), element[1])),
                     element[0].split('/')[0]))
        shutil.move(os.path.join(base_path, element[0]),
                    os.path.join(os.path.join(os.path.join(base_path, element[2]), element[1]), element[0]))

In [7]:
#only need to run once after unzipping images
process_folders()

In [3]:
def create_dict_bboxes(list_all, n = None,  split='train'):
    if n:
        list_all = list_all[:n]
    lst = [(line[0], line[1], line[3], line[2]) for line in list_all if line[2] == split]
    lst = [("".join(line[0].split('/')[0] + '/' + line[3] + '/' + line[1] + line[0][3:]), line[1], line[2]) for line in lst]
    lst_shape = [cv2.imread(create_dict_bboxes_path + line[0]).shape for line in lst]
    lst_norm = [(line[0], line[1], (round(line[2][0] / shape[1], 2), round(line[2][1] / shape[0], 2)
                                    , round(line[2][2] / shape[1], 2), round(line[2][3] / shape[0], 2)),shape) for line, shape in zip(lst, lst_shape)]
    dict_ = {"/".join(line[0].split('/')[2:]): {'x1': line[2][0], 'y1': line[2][1]
                                                , 'x2': line[2][2], 'y2': line[2][3], 'shape': line[3]} for line in lst_norm}
    return dict_

In [4]:
def get_dict_bboxes(n = None):
    with open('/home/ec2-user/data/deepfashion/anno/list_category_img.txt', 'r') as category_img_file, \
            open('/home/ec2-user/data/deepfashion/eval/list_eval_partition.txt', 'r') as eval_partition_file, \
            open('/home/ec2-user/data/deepfashion/anno/list_bbox.txt', 'r') as bbox_file:
        list_category_img = [line.rstrip('\n') for line in category_img_file][2:]
        list_eval_partition = [line.rstrip('\n') for line in eval_partition_file][2:]
        list_bbox = [line.rstrip('\n') for line in bbox_file][2:]

        list_category_img = [splitter.split(line) for line in list_category_img]
        list_eval_partition = [splitter.split(line) for line in list_eval_partition]
        list_bbox = [splitter.split(line) for line in list_bbox]

        list_all = [(k[0], k[0].split('/')[1].split('_')[-1], v[1], (int(b[1]), int(b[2]), int(b[3]), int(b[4])))
                    for k, v, b in zip(list_category_img, list_eval_partition, list_bbox)]
        
        list_all.sort(key=lambda x: x[1])

        dict_train = create_dict_bboxes(list_all, n)
        dict_val = create_dict_bboxes(list_all, n, split='val')
        dict_test = create_dict_bboxes(list_all, n, split='test')

        return dict_train, dict_val, dict_test

In [2]:
class DirectoryIteratorWithBoundingBoxes(DirectoryIterator):
    def __init__(self, directory, image_data_generator, bounding_boxes: dict = None, target_size=(256, 256),
                 color_mode: str = 'rgb', classes=None, class_mode: str = 'categorical', batch_size: int = 32,
                 shuffle: bool = True, seed=None, data_format=None, save_to_dir=None,
                 save_prefix: str = '', save_format: str = 'jpeg', follow_links: bool = False):
        super().__init__(directory, image_data_generator, target_size, color_mode, classes, class_mode, batch_size,
                         shuffle, seed, data_format, save_to_dir, save_prefix, save_format, follow_links)
        self.bounding_boxes = bounding_boxes

    def next(self):
        """
        # Returns
            The next batch.
        """
        with self.lock:
            index_array = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel
        batch_x = np.zeros((len(index_array),) + self.image_shape, dtype=K.floatx())
        locations = np.zeros((len(batch_x),) + (4,), dtype=K.floatx())

        # build batch of image data
        for i, j in enumerate(index_array):
            fname = self.filenames[j]
            img = image.load_img(os.path.join(self.directory, fname),
                                 target_size=self.target_size)
            x = image.img_to_array(img, data_format=self.data_format)
            x = preprocess_input(x)
            x = self.image_data_generator.random_transform(x)
            x = self.image_data_generator.standardize(x)
            batch_x[i] = x

            if self.bounding_boxes is not None:
                bounding_box = self.bounding_boxes[fname]
                locations[i] = np.asarray(
                    [bounding_box['x1'], bounding_box['y1'], bounding_box['x2'], bounding_box['y2']],
                    dtype=K.floatx())
        # optionally save augmented images to disk for debugging purposes
        # build batch of labels
        if self.class_mode == 'sparse':
            batch_y = self.classes[index_array]
        elif self.class_mode == 'binary':
            batch_y = self.classes[index_array].astype(K.floatx())
        elif self.class_mode == 'categorical':
            batch_y = np.zeros((len(batch_x), 46), dtype=K.floatx())
            for i, label in enumerate(self.classes[index_array]):
                batch_y[i, label] = 1.
        else:
            return batch_x

        if self.bounding_boxes is not None:
            return batch_x, [batch_y, locations]
        else:
            return batch_x, batch_y

In [4]:
model_resnet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [3]:
model_resnet = ResNet50(weights='imagenet', include_top=False, pooling='avg')

for layer in model_resnet.layers[:-12]:
    # 6 - 12 - 18 have been tried. 12 is the best.
    layer.trainable = False
    
x = model_resnet.output
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
y = Dense(46, activation='softmax', name='img')(x)

x_bbox = model_resnet.output
x_bbox = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x_bbox)
x_bbox = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x_bbox)
bbox = Dense(4, kernel_initializer='normal', name='bbox')(x_bbox)

final_model = Model(inputs=model_resnet.input,
                    outputs=[y, bbox])

opt = SGD(lr=0.0001, momentum=0.9, nesterov=True)

final_model.compile(optimizer=opt,
                    loss={'img': 'categorical_crossentropy',
                          'bbox': 'mean_squared_error'},
                    metrics={'img': ['accuracy', 'top_k_categorical_accuracy'], # default: top-5
                             'bbox': ['mse']})

W0922 00:05:58.521492 140507912988480 deprecation_wrapper.py:119] From /home/ec2-user/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0922 00:05:58.536009 140507912988480 deprecation_wrapper.py:119] From /home/ec2-user/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0922 00:05:58.540123 140507912988480 deprecation_wrapper.py:119] From /home/ec2-user/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0922 00:05:58.566353 140507912988480 deprecation_wrapper.py:119] From /home/ec2-user/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.c

In [18]:
#dict_train, dict_val, dict_test = get_dict_bboxes()

In [9]:
import json

json_train = json.dumps(dict_train)
f = open("/home/ec2-user/GitHub/deepfashion_keras/data/img_dicts/dict_train.json","w")
f.write(json_train)
f.close()

json_val = json.dumps(dict_val)
f = open("/home/ec2-user/GitHub/deepfashion_keras/data/img_dicts/dict_val.json","w")
f.write(json_val)
f.close()

json_test = json.dumps(dict_test)
f = open("/home/ec2-user/GitHub/deepfashion_keras/data/img_dicts/dict_test.json","w")
f.write(json_test)
f.close()

NameError: name 'dict_train' is not defined

In [12]:
with open('/ebsvol2/data/deepfashion_keras/data/img_dicts/dict_train.json', 'r') as train_f:
    dict_train = json.load(train_f)

with open('/ebsvol2/data/deepfashion_keras/data/img_dicts/dict_val.json', 'r') as val_f:
    dict_val = json.load(val_f)

with open('/ebsvol2/data/deepfashion_keras/data/img_dicts/dict_test.json', 'r') as test_f:
    dict_test = json.load(test_f)

In [16]:
train_dir = "/ebsvol2/data/deepfashion_keras/data/img/train"
val_dir = "/ebsvol2/data/deepfashion_keras/data/img/val"

train_datagen = ImageDataGenerator(rotation_range=30.,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True)
val_datagen = ImageDataGenerator()

train_iterator = DirectoryIteratorWithBoundingBoxes(train_dir, train_datagen, bounding_boxes=dict_train, target_size=(200, 200))
val_iterator = DirectoryIteratorWithBoundingBoxes(val_dir, val_datagen, bounding_boxes=dict_val,target_size=(200, 200))

Found 209222 images belonging to 46 classes.
Found 40000 images belonging to 46 classes.


In [21]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               patience=12,
                               factor=0.5,
                               verbose=1)
tensorboard = TensorBoard(log_dir='/home/ec2-user/GitHub/deepfashion_keras/tb_logs')
early_stopper = EarlyStopping(monitor='val_loss',
                              patience=30,
                              verbose=1)
performance_log = CSVLogger('/home/ec2-user/GitHub/deepfashion_keras/csv_logs/model_10_epoch_log.csv',separator=',',append=False)
checkpoint = ModelCheckpoint('/ebsvol2/data/deepfashion_keras/models/model_10_epoch.h5')

In [22]:
def custom_generator(iterator):
    while True:
        batch_x, batch_y = iterator.next()
        yield (batch_x, batch_y)

In [ ]:
final_model.fit_generator(custom_generator(train_iterator),
                          steps_per_epoch= 6538, # 209222 train records / batch size of 32
                          epochs=10, 
                          validation_data=custom_generator(val_iterator),
                          validation_steps= 1250, # 40000 val records/ batch size of 32
                          verbose=2,
                          shuffle=True,
                          callbacks=[lr_reducer, checkpoint, early_stopper, tensorboard, performance_log],
                          workers=12
                         ,use_multiprocessing=True)

In [18]:
test_datagen = ImageDataGenerator()

test_dir = "/home/ec2-user/GitHub/deepfashion_keras/data/img/test"


test_iterator = DirectoryIteratorWithBoundingBoxes(test_dir, test_datagen, bounding_boxes=dict_test, target_size=(200, 200))
scores = final_model.evaluate_generator(custom_generator(test_iterator), steps=1250)

print('Multi target loss: ' + str(scores[0]))
print('Image loss: ' + str(scores[1]))
print('Bounding boxes loss: ' + str(scores[2]))
print('Image accuracy: ' + str(scores[3]))
print('Top-5 image accuracy: ' + str(scores[4]))
print('Bounding boxes error: ' + str(scores[5]))

Found 40000 images belonging to 46 classes.
Multi target loss: 3.265303009033203
Image loss: 1.8043490158081055
Bounding boxes loss: 0.033604979045689105
Image accuracy: 0.49215
Top-5 image accuracy: 0.82965
Bounding boxes error: 0.033604979045689105


In [2]:
scores

NameError: name 'scores' is not defined